In [1]:
import cv2 
import csv
import glob
import natsort
import numpy as np
import re
import pickle

In [2]:
def get_numbers_from_filename(filename):
    return re.search(r'\d+', filename).group(0)

In [21]:
def markSafeDurationForVideo(video_path):

    # fore resizing a window for video (optional depending on screen size)
    cv2.namedWindow(video_path, cv2.WINDOW_AUTOSIZE)
    #cv2.resizeWindow(video_path, height=760, width=1366)

    # setting font properties for displaying frame number with video
    font = cv2.FONT_HERSHEY_COMPLEX  # fontFace
    fontScale = 1  # fontScale
    fontColor = (255,255,255) # fontColor
    lineType = 2 # lineType
    
    # capture a video from given path
    scanned_video = cv2.VideoCapture(video_path)
    no_frames = int(scanned_video.get(cv2.CAP_PROP_FRAME_COUNT))
    
    frame_count = -1
    flag = 1

    ss = []  # safe duration start list
    se = [] # safe duration end list
    safe_duration_list = [] # final list containing ss and se

    if (scanned_video.isOpened()== False): # check if there is error in opening a video
        raise Exception("check the videopath or error opening video")

    while(scanned_video.isOpened()):

        ret, frame = scanned_video.read() 
        key = cv2.waitKey(34)
        
        if(ret == True):

            if(frame_count == -1):
                while True:

                    key_init = cv2.waitKey(25)
                    cv2.putText(frame, "Frame Number:" + str(frame_count), org=(20,50), fontFace=font, fontScale=fontScale, color=fontColor, lineType=lineType, thickness=3)
                    cv2.imshow(video_path, frame)

                    if key_init == ord('a'):
                        break
            
            frame_count = frame_count + 1
            
            cv2.putText(frame, "Frame Number:" + str(frame_count), org=(20,50), fontFace=font, fontScale=fontScale, color=fontColor, lineType=lineType, thickness=3)
            cv2.imshow(video_path,frame)

            if key == ord('p'): # pause a video

                while True:
                    
                    key2 = cv2.waitKey(25)

                    cv2.putText(frame, "Frame Number:" + str(frame_count), org=(20,50), fontFace=font, fontScale=fontScale, color=fontColor, lineType=lineType, thickness=3)
                    cv2.imshow(video_path, frame)
                    
                    if key2 == ord('p'): # resume video after pausing
                        break
            
            if key == ord('0'): # save safe duration start

                ss.append(0)
                if (len(ss) != 1):
                    print("You have made some mistake")
                    flag = 0
                    break
                    
                safe_duration_list.append(0)
            
            if key == ord('s'): # save safe duration start

                ss.append(frame_count)
                if (len(ss) != len(se) + 1):
                    print("You have made some mistake")
                    flag = 0
                    break
                    
                safe_duration_list.append(frame_count)
                
            if key == ord('e'): # save safe duration end

                se.append(frame_count)
                if (len(se) != len(ss)):
                    print("You have made some mistake")
                    flag = 0
                    break
                safe_duration_list.append(frame_count)
            
            if key == ord('q'): #quit a video
                break

        else:
            break


    scanned_video.release()
    cv2.destroyAllWindows()

    if(len(safe_duration_list) % 2 !=0):
        safe_duration_list.append(no_frames - 1)
    
    return safe_duration_list, flag

In [22]:

def save_labels_csv(videos_list, csv_file, pickle_list_path):
    
    csv_writer = csv.writer(open(csv_file,"w"))
    open_file = open(pickle_list_path, "rb")
    pickle_list = pickle.load(open_file)
    open_file.close()
    
    for video_path in videos_list:
        
        print('\nstarting for', video_path)
        video_no = int(get_numbers_from_filename(video_path))
        ready_flag = str.lower(input('type y if you are ready to annotate the video and n to quit the program\n'))
        if(ready_flag == 'y'):
            correct_flag = 1
            surity_flag = 0
            while(surity_flag == 0 or correct_flag == 0):
                safe_duration_list, correct_flag = markSafeDurationForVideo(video_path)
                if(correct_flag == 0):
                    print("press keys carefully this time!")
                    input("press any key when you are ready")
                else:
                    surity_flag = int(input('Enter 0 to repeat the annotations process and 1 to proceed to next video\n'))
                if(surity_flag == 1 and correct_flag == 1):
                    break
            #csv_writer.writerow(safe_duration_list)
            pickle_list[video_no-1] = safe_duration_list

            print("safe_duration_list", safe_duration_list)
            print('------------------------------------------')
            
        else:
            
            print('\nTake a break! You must be tired')
            break
            
    open_file = open(pickle_list_path, "wb")
    pickle.dump(pickle_list, open_file)
    open_file.close()
    csv_writer.writerows(pickle_list)
    print("list and csv file saved")

In [23]:
video_folder = '/home/siddhi/Desktop/RoadCrossingAssistant_FY_Project_Data/videos_train/'
video_list = glob.glob(video_folder + '/video*.MOV')
video_list = (natsort.natsorted(video_list))
pickle_list_path = '/home/siddhi/Desktop/RoadCrossingAssistant_FY_Project_Data/labels_framewise.pkl'
# video_list = ['/home/yagnesh/Study/Machine Learning/ML projects/RoadCrossingAssistant_Arrays/videos_train/video1.MOV']

csv_file = '/home/siddhi/Desktop/RoadCrossingAssistant_FY_Project_Data/labels-frame-wise.csv'

save_labels_csv(video_list, csv_file, pickle_list_path)


starting for /home/siddhi/Desktop/RoadCrossingAssistant_FY_Project_Data/videos_train/video1.MOV
safe_duration_list []
------------------------------------------

starting for /home/siddhi/Desktop/RoadCrossingAssistant_FY_Project_Data/videos_train/video2.MOV
You have made some mistake
press keys carefully this time!
safe_duration_list [0, 16, 53, 149]
------------------------------------------

starting for /home/siddhi/Desktop/RoadCrossingAssistant_FY_Project_Data/videos_train/video3.MOV


ValueError: invalid literal for int() with base 10: ''

In [93]:
#labels_framewise = [[] for i in range(104)]
file_name = "/home/siddhi/Desktop/RoadCrossingAssistant_FY_Project_Data/labels_framewise.pkl"

#open_file = open(file_name, "wb")
#pickle.dump(labels_framewise, open_file)
#open_file.close()

open_file = open(file_name, "rb")
loaded_list = pickle.load(open_file)
open_file.close()

print(len(loaded_list),'\n', loaded_list)

104 
 [[], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], []]
